# 27) Generating Gaussians

In [118]:
import numpy as np 
from scipy import stats

## Auxillary Functions 

In order of appearance: 

1) Importance sampling (to generate the weights) <br> 
2) Sample Variance (for checking implementations and for later part) <br>
3) Sample Mean (for checking implementations) 

In [119]:
def importance_sampling_gauss(n, sigma): 
    '''
    params: 
        n - int, number of samples to generate 
        sigma - int, standard deviation 
    returns: 
        np.array of length n 
    '''
    x = np.random.normal(size = n)
    pi_t_y = stats.norm.pdf(x, scale = sigma)
    pi_y = stats.norm.pdf(x)
    
    w = pi_y / pi_t_y 
    w_norm = w / np.sum(w) 
    return np.array( [x, w_norm] )


def calc_sample_var_multi(X):
    ''' 
    Does what title says. Assumes expected values are known. 
    params: 
        X - 2d (n x p) np.matrix with X[:, j] as samples from distribution X_j 
    return: 
        vars - array-like of length p representing estimated variances for each 
    '''
    p = X.shape[1] #num columns 
    var = []
    for j in range(p): 
        resamples_j = np.squeeze(np.asarray(X[:, j]))
        var.append( np.var(resamples_j))
    return var

def calc_sample_average(X): 
    '''
    params: 
        X - nxp np.matrix where column j is sampled from distribution j 
    returns: 
        x_bar - np array of length p where each element j corresponds to mean of column j of X 
    '''
    p = X.shape[1] #num columns 
    x_bar = []
    for j in range(p): 
        resamples_j = np.squeeze(np.asarray(X[:, j]))
        x_bar.append( np.mean(resamples_j))
    return x_bar
    

## Resampling Implementations 

The four used are <br>
1) Multinomial  
2) Bernoulli <br>
3) Stratified <br>
4) Systematic <br>

### Multinomial 

In [120]:
def resample_gauss_multinom(weights, l):
    '''
    Using multinomial resampling to sample copy numbers N_k from set W of size n.  
    params: 
        weights - np.array of length n, s.t. sum(weights) = 1
        l - int, number of times to resample (for calculating variances I suppose) 
            Note that each l means sampling n * l times from the multinomial distribution 
    returns: 
        np.matrix n x l where each index [i,j] contains copy number sample i of sample x_j with weight w_j 
    '''
    resamples = []
    for i in range(l):
        resamples.append( np.random.multinomial( len(weights), weights) ) 
    return np.matrix( resamples )  


In [121]:
n_samples = 10
sigma = 2
X = importance_sampling_gauss(n_samples, 2) 
weights = X[1, :] 
expected = weights * n_samples #the expected value of the copy numbers given the weights, E[N_k]

l = 2000 #number of draws from resampling scheme distribution 
w_resample = resample_gauss_multinom(weights, l) #N_k  

In [122]:
mu = n_samples * weights 
est_var_multinom = calc_sample_var_multi(mu = mu, X = w_resample)
var_multinom = n_samples * weights * ( 1 - weights)
print(var_multinom)
print(est_var_multinom)

TypeError: calc_sample_var_multi() got an unexpected keyword argument 'mu'

### Bernoulli Resampling) 

In [ ]:
def resample_bernoulli(weights, l): 
    '''
    Generates new copy numbers N_k using Bernoilli Resampling technique 
    params: 
        weights - np array
        l - number of draws 
    returns: 
         np.matrix n x l where each index [i,j] contains copy number sample i of sample x_j with weight w_j 
    '''
    n = len(weights)
    NW = n * weights
    NW_floor = np.floor(NW)
    
    resamples = [] 
    for i in range(l):
        u = np.random.uniform(size = n)
        cond = (u <  NW - NW_floor)
        resamples.append( NW_floor + cond ) 
    return np.matrix( resamples ) 

In [ ]:
w_resample_bernoulli = resample_bernoulli(weights, l)
print(calc_sample_var_multi(mu = mu, X = w_resample_bernoulli))
var_resample_bernoulli = (mu - np.floor(mu)) * (np.ceil(mu) - mu)
print(var_resample_bernoulli)

### Stratified Resampling 

In [ ]:
def get_ranges(weights): 
    '''
    params: 
        weights - np array of length n representing sample weights 
    returns: 
        np array of length n where each element i contains sum(weights_j) for j <= i
    '''    
    thresholds = [] 
    cur = 0 
    for w in weights:
        cur += w
        thresholds.append(cur)
    return thresholds 

def count_points(thresholds, u): 
    '''
    params: 
       thresholds - np array of length n representing cumulative sums of weights up to element i 
       u - sorted np array of length, presumably generated from stratas 
    returns: 
        np array of length n where each element i represents the copy count, 
            here it is the number of points that fall between 
            thresholds[i - 1] and thresholds[i]
    '''
    j = 0 #iterator through u  
    N_k = [] 
    count = 0 
    for bound in thresholds: 
        #print("bound: {}".format(bound))
        while j < len(u): 
            #print("u[j]: {}".format(u[j]))
            if bound > u[j]: 
                #print("Yes! u[j] ({}) < bound ({})".format(u[j], bound))
                count += 1
                j += 1 
            else: 
                #print("No. u[j] ({}) > bound ({})".format(u[j], bound))
                N_k.append(count)
                count = 0
                break

    #print("count after: {}".format( count) ) 
    #now we need to fill in rest of the spots in N_k 
    N_k.append(count)
    while len(N_k) < len(u): 
        N_k.append(0)
    return N_k 


def resample_stratified(weights, l): 
    '''
    Generates new copy numbers N_k using Stratified Resampling technique 
    params: 
        weights - np array 
        l - number of draws 
    returns: 
         np.matrix n x l where each index [i,j] contains copy number sample i of sample x_j with weight w_j 
    '''
    thresholds = get_ranges(weights)
        
    resamples = [] 
    n = len(weights)
    for i in range(l):
        u = []
        for j in range(n): #draw uniformally from each strata 
            u.append( np.random.uniform(low = j / n, high = (j + 1) / n) ) 
        N_i = count_points(thresholds, u)
        
        resamples.append( N_i ) 
    #print(len(resamples))
    #print(len(N_i))
    return np.matrix( resamples ) 

In [ ]:
thresholds = get_ranges(weights)
u = []
for j in range(n_samples): #draw uniformally from each strata 
    u.append( np.random.uniform(low = j / n_samples, high = (j + 1) / n_samples) ) 
N_i = count_points(thresholds, u)
print("thresholds: {}".format( np.round( thresholds, decimals=3) )  )
print("u: {}".format( np.round( u, decimals=3) ) )
print(N_i, len(N_i))
print("weights: {}".format(np.round(weights, decimals = 3)))

In [ ]:
resample_strata = resample_stratified(weights, l)
#print(resample_strata.shape)
strata_xbars = calc_sample_average(resample_strata)

#print(strata_xbars - expected)
#print(expected)

### Systematic Resampling 

In [ ]:
def resample_systematic(weights, l): 
    '''
    Generates new copy numbers N_k using the Systematic Resampling technique 
    params: 
        weights - np array
        l - number of draws 
    returns: 
         np.matrix n x l where each index [i,j] contains copy number sample i of sample x_j with weight w_j 
    '''
    n = len(weights)
    thresholds = get_ranges(weights)
    intervals = np.array( range(n) ) / n
    
    resamples = [] 
    
    for i in range(l):
        u = np.random.uniform(high = 1 / n ) + intervals
        N_i = count_points(thresholds, u)
        resamples.append( N_i ) 
    return np.matrix( resamples )     

In [124]:
resample_sys = resample_systematic(weights, l)
resample_sys.shape
#sys_xbar = calc_sample_average(resample_sys)
#print( sys_xbar - expected ) 

(2000, 10)

## Observing the sample variances 

We shall test the sample variances for l = 2000 resamples for n = 10 Importance samples (So we are actually drawing 2000 * 10 in total) for $\sigma =  \{1, 1.5, 2, 4, 8, 16\}$


In [127]:
sigmas = [1, 1.5, 2, 4, 8, 16]
n_samples = 10
l = 2000 #number of draws from resampling scheme distribution


variances = {'multinom' : [], 'bernoulli': [], 'stratified' : [], 'systematic' : []} 
for sigma in sigmas: 
    X = importance_sampling_gauss(n_samples, 2) 
    weights = X[1, :] 
    mu = weights * n_samples #the expected value of the copy numbers given the weights, E[N_k]
    
    multinom_resample = resample_gauss_multinom(weights, l) #N_k
    multinom_vars = calc_sample_var_multi(multinom_resample)
    variances['multinom'].append(multinom_vars)
    
    bernoulli_resample = resample_bernoulli(weights, l)
    bernoulli_vars = calc_sample_var_multi(bernoulli_resample)
    variances['bernoulli'].append(bernoulli_vars)

    stratified_resample = resample_stratified(weights, l)
    stratified_vars = calc_sample_var_multi(stratified_resample)
    variances['stratified'].append(stratified_vars)

    sys_resample = resample_systematic(weights, l)
    sys_vars = calc_sample_var_multi(sys_resample)
    variances['systematic'].append(sys_vars)


In [ ]:
a = [1]
a.append([1,2])
a